<a href="https://colab.research.google.com/github/BechirMETTALI/ProjectsRep/blob/master/XGBfocal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PREDICTING SEX FROM BRAIN RYTHM




*XGBOOST with Focal Loss function*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install imbalance-xgboost

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
import h5py
from keras.utils.np_utils import to_categorical
from imblearn.under_sampling import ClusterCentroids
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb


Loading Data

In [ ]:
path_X_train_new = '/content/drive/MyDrive/gcha/X_train_new.h5'
path_X_test_new = '/content/drive/MyDrive/gcha/X_test_new.h5'
path_y_train_AvC = '/content/drive/MyDrive/gcha/y_train_AvCsavx.csv'
file_x_train = h5py.File(path_X_train_new, 'r')
x_train = file_x_train['features']
x_train_shape = x_train.shape 
print(x_train_shape)
file_x_test = h5py.File(path_X_test_new, 'r')
x_test = file_x_test['features']
x_test_shape = x_test.shape 
print(x_test_shape)


(946, 40, 7, 500)
(946, 40, 7, 500)


In [ ]:
n_d = x_train_shape[0]
n_canaux = x_train_shape[1]
n_channels = x_train_shape[2]
n_signal = x_train_shape[3]
num_channel = 1
x_train_array = x_train[0:n_d, 0:n_canaux, 0:n_channels, 0:n_signal]

x_train_xgb=x_train_array.reshape(n_d * n_canaux , n_channels * n_signal)
print (x_train_xgb.shape)
n_dt = x_test_shape[0]
x_test_array = x_test[0:n_dt, 0:n_canaux, 0:n_channels, 0:n_signal]
x_test_xgb=x_test_array.reshape(n_dt * n_canaux , n_channels * n_signal)
print (x_test_xgb.shape)

(37840, 3500)
(37840, 3500)


In [ ]:
y_train_import = np.genfromtxt(path_y_train_AvC , dtype="str" , delimiter=" " )
excel_0 = y_train_import[0]  
y_train_str = y_train_import[1:y_train_import.shape[0]]
y_train = np.array([np.int(identity[-1]) for identity in y_train_str for i in range(n_canaux)] )  #0 male 1 female



print (y_train )

[0 0 0 ... 0 0 0]


In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
xgbooster = imb_xgb(special_objective='focal')
CV_focal_booster = GridSearchCV(xgbooster, {"focal_gamma":[1.0,1.5,2.0,2.5,3.0]})
CV_focal_booster.fit(x_train_xgb, y_train)
opt_focal_booster = CV_focal_booster.best_estimator_
preds = opt_focal_booster.predict_two_class(x_test_xgb) 
#clf = xgb.XGBClassifier(n_estimators = 20,scale_pos_weight=2.3)
#clf.fit(x_train_xgb, y_train)
#preds = clf.predict(x_test_xgb)
#acc_xgb = (preds == Y_test).sum().astype(float) / len(preds)*100
print(sum(preds))


[24419.764 13420.149]


In [ ]:
y_test = [preds[n_canaux*i] for i in range(n_d) ]
print(y_test)
y_test_final=[]
for i in range (len(y_test)):
  if (y_test[i][0] > 0.5) :
    y_test_final.append(0)
  else:
    y_test_final.append(1)
print(len(y_test_final))
print(len(y_test))
y_solution = [excel_0] + [np.str(i)+','+np.str(y_test_final[i]) for i in range (len(y_test_final))]
print(y_solution)
np.savetxt("Y_foc.csv", np.array(y_solution), delimiter=" ", fmt='%s')

[array([0.6674473 , 0.33255264], dtype=float32), array([0.64191806, 0.35808194], dtype=float32), array([0.46467602, 0.535324  ], dtype=float32), array([0.6303551 , 0.36964488], dtype=float32), array([0.47112042, 0.5288796 ], dtype=float32), array([0.624377, 0.375623], dtype=float32), array([0.67632365, 0.32367632], dtype=float32), array([0.6667098 , 0.33329025], dtype=float32), array([0.71509856, 0.28490144], dtype=float32), array([0.6242105, 0.3757895], dtype=float32), array([0.7067051 , 0.29329494], dtype=float32), array([0.7475925, 0.2524075], dtype=float32), array([0.6900885 , 0.30991152], dtype=float32), array([0.63460743, 0.36539257], dtype=float32), array([0.7060713 , 0.29392868], dtype=float32), array([0.68990445, 0.31009555], dtype=float32), array([0.6482092 , 0.35179082], dtype=float32), array([0.57408035, 0.42591962], dtype=float32), array([0.7921803 , 0.20781972], dtype=float32), array([0.70157504, 0.298425  ], dtype=float32), array([0.6435127 , 0.35648724], dtype=float32),